# Simple RAG

Simple RAG 구조는 다음과 같은 단계로 구성된다.:

1. 데이터 수집 (Data Ingestion): 텍스트 데이터를 불러오고 전처리한다.
1. 청크 분할 (Chunking): 검색 성능을 높이기 위해 데이터를 작은 단위로 나눈다.
1. 임베딩 생성 (Embedding Creation): 텍스트 청크를 임베딩 모델을 이용해 수치 벡터로 변환한다.
1. 의미 기반 검색 (Semantic Search): 사용자 질의에 기반하여 의미적으로 관련 있는 청크를 검색한다.
1. 응답 생성 (Response Generation): 검색된 문맥을 바탕으로 언어 모델이 응답을 생성한다.

In [ ]:
%pip install openai 
%pip install fitz
%pip install python-dotenv

## Setting Up the Environment

In [1]:
import fitz
import numpy as np
import json

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

import os
from dotenv import load_dotenv  

!echo "발급 받은 OpenAI API Key를 여기에 붙혀넣으세요." >> .env
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Extracting Text from a PDF File

In [3]:
# PDF 파일에서 텍스트를 추출하는 함수
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 전체 텍스트를 추출합니다.

    Args:
        pdf_path (str): PDF 파일 경로

    Returns:
        str: 추출된 전체 텍스트
    """
    # PDF 파일 열기
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 추출된 텍스트를 저장할 문자열 초기화

    # 각 페이지를 순회하며 텍스트 추출
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # 해당 페이지 가져오기
        text = page.get_text("text")  # 텍스트 추출
        all_text += text  # 누적하여 전체 텍스트에 추가

    return all_text  # 최종적으로 전체 텍스트 반환

## Chunking the Extracted Text

In [4]:
from typing import List

# 텍스트를 일정 길이(n)와 중첩(overlap)을 기준으로 나누는 함수
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 n자 단위로, 지정된 overlap만큼 겹치도록 분할합니다.

    Args:
        text (str): 분할할 원본 텍스트
        n (int): 각 청크(chunk)의 문자 수
        overlap (int): 청크 간 겹치는 문자 수

    Returns:
        List[str]: 나누어진 텍스트 청크들의 리스트
    """
    chunks = []  # 청크를 저장할 빈 리스트 초기화

    # (n - overlap) 간격으로 텍스트를 순회하며 청크를 생성
    for i in range(0, len(text), n - overlap):
        # i부터 i + n까지의 텍스트를 청크로 잘라서 추가
        chunks.append(text[i:i + n])

    return chunks  # 청크 리스트 반환

## Extracting and Chunking Text from a PDF File

In [5]:
# PDF 파일 경로 지정
pdf_path = "dataset/AI_Understanding.pdf"

# PDF 파일에서 텍스트 추출
extracted_text = extract_text_from_pdf(pdf_path)

# 추출한 텍스트를 1000자 단위로, 200자 중첩하여 청크로 분할
text_chunks = chunk_text(extracted_text, 1000, 200)

# 생성된 청크 수 출력
print("텍스트 청크 개수:", len(text_chunks))

# 첫 번째 청크 내용 출력
print("\n첫 번째 텍스트 청크:")
print(text_chunks[0])

텍스트 청크 개수: 21

첫 번째 텍스트 청크:
인공 지능 이해 
 
1장: 인공 지능 소개 
인공 지능(AI)은 디지털 컴퓨터 또는 컴퓨터로 제어되는 로봇이 지적인 존재와 일반적으로 
관련된 작업을 수행할 수 있는 능력을 말합니다. 이 용어는 추론, 의미 발견, 일반화, 과거 
경험으로부터의 학습 능력 등 인간의 특징적인 지적 프로세스가 부여된 시스템을 
개발하는 프로젝트에 자주 적용됩니다. 지난 수십 년 동안 컴퓨팅 성능과 데이터 가용성의 
발전으로 AI의 개발과 배포가 크게 가속화되었습니다. 
역사적 맥락 
인공 지능에 대한 개념은 수세기 동안 존재해 왔으며 종종 신화와 소설에 묘사되기도 
했습니다. 하지만 공식적인 AI 연구 분야는 20세기 중반에 시작되었습니다. 1956년 
다트머스 워크숍은 AI의 발상지로 널리 알려져 있습니다. 초기 AI 연구는 문제 해결과 
상징적 방법에 중점을 두었습니다. 1980년대에는 전문가 시스템이 등장했고, 1990년대와 
2000년대에는 머신러닝과 신경망이 발전했습니다. 최근 딥러닝의 획기적인 발전은 이 
분야에 혁신을 가져왔습니다. 
현대 관측 
최신 AI 시스템은 일상 생활에서 점점 더 널리 보급되고 있습니다. Siri와 Alexa 같은 가상 
비서부터 스트리밍 서비스 및 소셜 미디어의 추천 알고리즘에 이르기까지 AI는 우리의 
생활, 업무, 상호 작용 방식에 영향을 미치고 있습니다. 자율 주행 자동차, 첨단 의료 진단, 
정교한 재무 모델링 도구의 개발은 AI가 광범위하고 성장하는 응용 분야를 보여줍니다. 
윤리적 영향, 편견, 일자리 대체에 대한 우려도 점점 더 커지고 있습니다. 
 
2장: 인공 지능의 핵심 개념 
머신 러닝 
머신러닝(ML)은 명시적으로 프로그래밍하지 않고도 시스템이 데이터로부터 학습할 수 
있도록 하는 데 중점을 둔 AI의 하위 집합입니다. ML 알고리즘은 더 많은 데이터에 
노출됨에 따라 패턴을 식별하고 예측하며 시간이 지남에 따라 성능을 개선합니다. 
지도 학습 
지도 학습에서는 입력 

## Creating Embeddings for Text Chunks

In [7]:
# 주어진 텍스트에 대해 지정된 모델을 사용하여 임베딩을 생성하는 함수
def create_embeddings(text, model="text-embedding-3-small"):
    """
    지정된 OpenAI 모델을 사용하여 주어진 텍스트에 대한 임베딩을 생성합니다.

    Args:
        text (str): 임베딩을 생성할 입력 텍스트
        model (str): 사용할 임베딩 모델 (기본값: "BAAI/bge-en-icl")

    Returns:
        dict: OpenAI API로부터 받은 임베딩 응답 결과
    """
    # 지정된 모델을 사용하여 텍스트 임베딩 생성
    response = client.embeddings.create(
        model=model,
        input=text
    )

    return response  # 임베딩 결과 응답 반환

# 분할된 텍스트 청크에 대해 임베딩 생성
response = create_embeddings(text_chunks)

## Performing Semantic Search

In [8]:
# 두 벡터 간 코사인 유사도를 계산하는 함수
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
        vec1 (np.ndarray): 첫 번째 벡터
        vec2 (np.ndarray): 두 번째 벡터

    Returns:
        float: 두 벡터 간의 코사인 유사도 (값의 범위: -1 ~ 1)
    """
    # 두 벡터의 내적을 계산하고, 각 벡터의 노름 곱으로 나눔
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [9]:
# 쿼리를 기반으로 텍스트 청크에서 의미 기반 검색을 수행하는 함수
def semantic_search(query, text_chunks, embeddings, k=5):
    """
    주어진 쿼리와 임베딩을 사용하여 텍스트 청크에서 의미 기반 검색을 수행합니다.

    Args:
        query (str): 의미 검색에 사용할 쿼리 텍스트
        text_chunks (List[str]): 검색 대상이 되는 텍스트 청크 리스트
        embeddings (List[dict]): 각 청크에 대한 임베딩 객체 리스트
        k (int): 상위 k개의 관련 텍스트 청크를 반환 (기본값: 5)

    Returns:
        List[str]: 쿼리와 가장 관련 있는 텍스트 청크 상위 k개
    """
    # 쿼리에 대한 임베딩 생성
    query_embedding = create_embeddings(query).data[0].embedding
    similarity_scores = []  # 유사도 점수를 저장할 리스트 초기화

    # 각 텍스트 청크의 임베딩과 쿼리 임베딩 간의 코사인 유사도 계산
    for i, chunk_embedding in enumerate(embeddings):
        similarity_score = cosine_similarity(
            np.array(query_embedding), 
            np.array(chunk_embedding.embedding)
        )
        similarity_scores.append((i, similarity_score))  # 인덱스와 유사도 함께 저장

    # 유사도 점수를 기준으로 내림차순 정렬
    similarity_scores.sort(key=lambda x: x[1], reverse=True)

    # 상위 k개의 청크 인덱스를 추출
    top_indices = [index for index, _ in similarity_scores[:k]]

    # 상위 k개의 관련 텍스트 청크를 반환
    return [text_chunks[index] for index in top_indices]

## Running a Query on Extracted Chunks

In [10]:
import json

# 검증용 JSON 파일을 불러옴
with open('../../dataset/validation.json') as f:
    data = json.load(f)

# 첫 번째 항목에서 질의 추출
query = data[0]['question']

# 의미 기반 검색 수행: 주어진 쿼리에 대해 가장 관련성 높은 텍스트 청크 2개 검색
top_chunks = semantic_search(query, text_chunks, response.data, k=2)

# 질의 출력
print("질의(Query):", query)

# 관련성 높은 문맥 청크 2개 출력
for i, chunk in enumerate(top_chunks):
    print(f"문맥 {i + 1}:\n{chunk}\n----------------")

질의(Query): '설명 가능한 AI(Explainable AI)'란 무엇이며, 왜 중요한가?
문맥 1:
고 사회에 유익한 AI 시스템의 개발과 
배포를 보장하기 위한 지침입니다. 주요 원칙에는 인권 존중, 개인정보 보호, 비차별, 
공익성이 포함됩니다. 
AI의 편향성 해결 
AI 시스템은 데이터에 존재하는 편견을 유전하고 증폭시켜 불공정하거나 차별적인 결과를 
초래할 수 있습니다. 편향성을 해결하려면 신중한 데이터 수집, 알고리즘 설계, 지속적인 
모니터링 및 평가가 필요합니다. 
투명성 및 설명 가능성 
투명성과 설명 가능성은 AI 시스템에 대한 신뢰를 구축하는 데 필수적입니다. 설명 가능한 
AI(XAI) 기술은 AI의 결정을 더 이해하기 쉽게 만들어 사용자가 공정성과 정확성을 
평가할 수 있도록 하는 것을 목표로 합니다. 
개인정보 및 데이터 보호 
AI 시스템은 대량의 데이터에 의존하는 경우가 많기 때문에 개인정보 보호와 데이터 
보호에 대한 우려가 제기됩니다. 책임감 있는 데이터 처리, 개인정보 보호 기술 구현, 
데이터 보호 규정 준수는 매우 중요합니다. 
책임과 의무 
AI 시스템에 대한 책임과 의무를 확립하는 것은 잠재적인 피해를 해결하고 윤리적 행동을 
보장하는 데 필수적입니다. 여기에는 AI 시스템의 개발자, 배포자, 사용자에 대한 역할과 
책임을 정의하는 것이 포함됩니다. 
 
20장: AI에 대한 신뢰 구축 
투명성 및 설명 가능성 
투명성과 설명 가능성은 AI에 대한 신뢰를 구축하는 데 있어 핵심입니다. AI 시스템을 
이해하기 쉽게 만들고 의사 결정 프로세스에 대한 인사이트를 제공하면 사용자가 AI의 
신뢰성과 공정성을 평가하는 데 도움이 됩니다. 
견고성 및 신뢰성 
AI 시스템의 견고성과 신뢰성을 확보하는 것은 신뢰를 구축하는 데 필수적입니다. 
여기에는 AI 모델 테스트 및 검증, 성능 모니터링, 잠재적인 취약점 해결이 포함됩니다. 
사용자 제어 및 대행사 
사용자에게 AI 시스템을 제어할 수 있는 권한을 부여하고 AI와의 

## Generating a Response Based on Retrieved Chunks

In [11]:
# AI 어시스턴트를 위한 시스템 프롬프트 정의
system_prompt = (
    "당신은 주어진 문맥을 기반으로만 답변하는 AI 어시스턴트입니다. "
    "제공된 문맥에서 직접적으로 답을 도출할 수 없는 경우에는 다음과 같이 답하십시오: "
    "'I do not have enough information to answer that.'"
)

# AI 응답을 생성하는 함수
def generate_response(system_prompt, user_message, model="gpt-4o-mini"):
    """
    시스템 프롬프트와 사용자 메시지를 기반으로 AI 모델의 응답을 생성합니다.

    Args:
        system_prompt (str): AI의 응답 방식을 지정하는 시스템 메시지
        user_message (str): 사용자 질의 또는 메시지
        model (str): 사용할 언어 모델 이름

    Returns:
        dict: 생성된 AI 응답을 포함한 API 응답 객체
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,  # 일관된 응답을 위해 temperature는 0
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# 검색된 상위 문맥(top_chunks)을 기반으로 사용자 프롬프트 구성
user_prompt = "\n".join([
    f"Context {i + 1}:\n{chunk}\n--------------\n" 
    for i, chunk in enumerate(top_chunks)
])
user_prompt = f"{user_prompt}\nQuestion: {query}"

# AI 응답 생성
ai_response = generate_response(system_prompt, user_prompt)

## Evaluating the AI Response

In [12]:
# 평가 시스템을 위한 시스템 프롬프트 정의
evaluate_system_prompt = (
    "당신은 AI 어시스턴트의 응답을 평가하는 지능형 평가 시스템입니다. "
    "AI 응답이 정답에 매우 근접하면 점수 1점을 부여하고, "
    "정답과 맞지 않거나 불만족스러우면 0점을 부여하세요. "
    "부분적으로 일치하면 0.5점을 부여하세요."
)

# 사용자 질의, AI 응답, 정답, 평가 프롬프트를 결합하여 평가용 프롬프트 생성
evaluation_prompt = (
    f"User Query: {query}\n"
    f"AI Response:\n{ai_response.choices[0].message.content}\n"
    f"True Response: {data[0]['ideal_answer']}\n"
    f"{evaluate_system_prompt}"
)

# 평가 시스템 프롬프트를 사용하여 AI 응답 평가 점수 생성
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# 평가 결과 출력
print(evaluation_response.choices[0].message.content)

1점
